In [3]:
from collections import defaultdict
from multiprocessing import Pool
from pathlib import Path

from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from consent.consistency.comply_util import site_to_contras
from consent.consistency.cookie_pref_match import cookie_pref_match
from consent.consistency.util import get_scan_root_dir, get_scan_dirs # , FIG_DIR uncomment to save fig.
from consent.data.pref_menu_scan.cookie_pref_reader import read_cookie_prefs_in_scans
from consent.data.pref_menu_scan.log_reader import read_logs_in_scans
from consent.data.pref_menu_scan.cookie_decl_reader import read_cookie_decls_in_scans
from consent.data.pref_menu_scan.har_cookie_reader import read_postrej_sent_cookies_in_scans
from ooutil.type_util import hashabledict


[PosixPath('/mnt/sda/ducbui/Dropbox/Dropbox (University of Michigan)/projects/data_sync/consent/2022-05-30/pref_menu_scan_0k_20k'),
 PosixPath('/mnt/sda/ducbui/Dropbox/Dropbox (University of Michigan)/projects/data_sync/consent/2022-05-30/pref_menu_scan_20k_40k'),
 PosixPath('/mnt/sda/ducbui/Dropbox/Dropbox (University of Michigan)/projects/data_sync/consent/2022-05-30/pref_menu_scan_40k_60k'),
 PosixPath('/mnt/sda/ducbui/Dropbox/Dropbox (University of Michigan)/projects/data_sync/consent/2022-05-30/pref_menu_scan_60k_80k'),
 PosixPath('/mnt/sda/ducbui/Dropbox/Dropbox (University of Michigan)/projects/data_sync/consent/2022-05-30/pref_menu_scan_80k_100k')]

In [4]:
# Get cookie prefs = cookie decls + prefs
# TODO: cache this ...


Error reading intel.com dictionary update sequence element #0 has length 1; 2 is required
Error reading copper.com dictionary update sequence element #0 has length 1; 2 is required
Error reading intel.co.uk dictionary update sequence element #0 has length 1; 2 is required
Fail to read log 3 sites: ['intel.com', 'copper.com', 'intel.co.uk'] 
https://cdn-ukwest.onetrust.com/consent/e4eb3b2e-4cc5-4752-b2ca-381970849659/204361df-8a07-4ca3-a82b-f9c3a0433d38/en.json WARNING: Cookie list has session cookie but duration > 0
https://cdn.cookielaw.org/consent/bfc3f6d1-88f6-4292-b5d2-757633a696fe/ac184959-dbdc-45be-bc47-5fe1f7f7a064/en.json WARNING: Cookie list has session cookie but duration > 0
https://cdn.cookielaw.org/consent/c570eb43-1169-4222-914c-38a09bd70ca0/a1f914ac-390b-484b-9577-7c7ad52c8361/en.json WARNING: Cookie list has session cookie but duration > 0
https://cdn.cookielaw.org/consent/c570eb43-1169-4222-914c-38a09bd70ca0/a1f914ac-390b-484b-9577-7c7ad52c8361/en.json WARNING: Cookie 

,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name,consent
0,__we_bucket_id,www.wework.com,365 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True
1,OptanonAlertBoxClosed,wework.com,365 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True
2,OptanonConsent,www.wework.com,365 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True
3,__we_request_id,www.wework.com,Session,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True
4,we_referring_domain,www.wework.com,14 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True


Written to /mnt/sda/ducbui/Dropbox/Dropbox (University of Michigan)/projects/data_sync/consent/2022-05-30/cookie_prefs.parquet


In [7]:
cookielist_sites = sorted(cookie_prefs.site.unique().tolist())
n_cookielist_sites = len(cookielist_sites)
assert cookie_prefs.site.nunique() == n_cookielist_sites
n=10
print("Sites with cookie prefs:", n_cookielist_sites, f', first {n} sites:' , cookielist_sites[:n])

Sites with cookie prefs: 3809 , first 10 sites: ['100percentpure.com', '101blockchains.com', '123helpme.com', '177milkstreet.com', '1hotels.com', '24hourfitness.com', '24s.com', '2k.com', '2kgames.com', '2ksports.com']


In [6]:
print("Found cookie libraries:")
cookie_prefs[['site', 'lib_name']].drop_duplicates().lib_name.value_counts()

Found cookie libraries:


onetrust     3352
cookiebot     433
termly         24
Name: lib_name, dtype: int64

In [ ]:
print(f"Number of extracted cookie preferences: {len(cookie_prefs):,d}")